<a href="https://colab.research.google.com/github/kty2253/big_data_project/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84%EA%B2%80%EC%83%89_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

네이버 검색 API를 이용하여 1000개를 추출하여 JSON 형식으로 저장.

In [ ]:
import os
import sys
import urllib.request
import datetime
import time
import json


client_id = ' '
client_secret = ' '


def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None


def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)

    url = base + node + parameters
    responseDecode = getRequestUrl(url)

    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)


def getPostData(post, jsonResult, cnt):
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']

    pDate = datetime.datetime.strptime(post['pubDate'],  '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    # Set your desired date range
    start_date = datetime.datetime.strptime('2023-01-01', '%Y-%m-%d')
    end_date = datetime.datetime.strptime('2023-12-31', '%Y-%m-%d')

    # Check if the date of the news falls into the range
    if start_date <= datetime.datetime.strptime(pDate, '%Y-%m-%d %H:%M:%S') <= end_date:
        jsonResult.append({'cnt':cnt, 'title':title, 'description': description,
        'org_link':org_link,   'link': org_link,   'pDate':pDate})
    return


def main():
    node = 'news'
    srcText = input('검색어를 입력하세요: ')
    cnt = 0
    jsonResult = []

    jsonResponse = getNaverSearch(node, srcText, 1, 100)
    total = jsonResponse['total']

    while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
        for post in jsonResponse['items']:
            cnt += 1
            getPostData(post, jsonResult, cnt)

        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100)

    print('전체 검색 : %d 건' %total)

    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult,  indent=4, sort_keys=True,  ensure_ascii=False)

        outfile.write(jsonFile)

    print("가져온 데이터 : %d 건" %(len(jsonResult)))
    print ('%s_naver_%s.json SAVED' % (srcText, node))

if __name__ == '__main__':
    main()

검색어를 입력하세요: 3기 신도시
[2023-12-03 07:35:07.573082] Url Request Success
[2023-12-03 07:35:08.743269] Url Request Success
[2023-12-03 07:35:10.046070] Url Request Success
[2023-12-03 07:35:11.239099] Url Request Success
[2023-12-03 07:35:12.429729] Url Request Success
[2023-12-03 07:35:13.644804] Url Request Success
[2023-12-03 07:35:14.832873] Url Request Success
[2023-12-03 07:35:16.030484] Url Request Success
[2023-12-03 07:35:17.228957] Url Request Success
[2023-12-03 07:35:18.418256] Url Request Success
HTTP Error 400: Bad Request
[2023-12-03 07:35:19.371387] Error for URL : https://openapi.naver.com/v1/search/news.json?query=3%EA%B8%B0%20%EC%8B%A0%EB%8F%84%EC%8B%9C&start=1001&display=100
전체 검색 : 124965 건
가져온 데이터 : 1000 건
3기 신도시_naver_news.json SAVED
